In [1]:
%load_ext autoreload

In [14]:
import sys

import altair as alt
import polars as pl

sys.path.append("../")
%autoreload
from altair_utils import color_weekday_weekend

In [31]:
df = (
    pl.read_csv(r"Q:\Model Research\downtown_today\muni-ridership-monthly.csv")
    .with_columns(
        date=pl.col("MONTH").str.split(" ").list.first().str.to_date("%m/%d/%Y"),
        weekday=(pl.col("Service Day of the Week") == "WEEKDAY"),
    )
    # get mean of Sat and Sun
    .group_by("date", "weekday")
    .agg(pl.mean("Average Daily Boardings").alias("average daily boardings"))
    # take annual average
    .with_columns(
        pl.col("weekday")
        .replace_strict({True: "weekday", False: "weekend"})
        .alias("day of week"),
    )
    .sort("date")
)

In [32]:
annual_df = (
    df.with_columns(year=pl.col("date").dt.year())
    .group_by("year", "day of week")
    .agg(pl.mean("average daily boardings"))
    .sort("year", "day of week")
    .filter((pl.col("year") > 2018) & (pl.col("year") < 2024))  # partial years
)
quarterly_df = (
    df.sort("date")
    .group_by_dynamic("date", every="3mo", group_by="day of week")
    .agg(pl.mean("average daily boardings"))
).with_columns(
    year=pl.col("date").dt.year(),
    quarter=pl.col("date").dt.quarter(),
)

In [ ]:
(
    alt.Chart(annual_df, title="Muni ridership (annual avg)")
    .mark_line()
    .encode(
        x="year",
        y="average daily boardings",
        color=alt.Color("day of week:N").scale(
            domain=["weekday", "weekend"], range=color_weekday_weekend
        ),
    )
)

In [ ]:
quarterly_df.write_csv("../output/data/muni-ridership-trend.csv")
chart = (
    alt.Chart(quarterly_df)
    .mark_line()
    .encode(
        x="date",
        y=alt.Y(
            "average daily boardings",
            title="Muni average daily boardings (quarterly)",
        ),
        color=alt.Color("day of week:N").scale(
            domain=["weekday", "weekend"], range=color_weekday_weekend
        ),
        tooltip=["year", "quarter", "day of week", "average daily boardings"],
    )
)
chart.save("../output/Links/muni-ridership-trend.png")
chart.save("../output/Links/muni-ridership-trend.html")
chart